In [ ]:
import json
import os
import random
from sklearn.model_selection import train_test_split

# Load your existing manifest
with open("../creolese-audio-dataset/transcripts.json", "r") as f:
    dataset = json.load(f)

# Convert to NeMo manifest format
nemo_manifest = []
for item in dataset:
    filename = os.path.basename(item["audio"])
    audio_path = os.path.join("./creolese-audio-dataset/Audio Files", filename)

    # Get audio duration
    import soundfile as sf

    def get_duration(audio_path):
        audio_data, sample_rate = sf.read(audio_path)
        duration = len(audio_data) / sample_rate
        return duration

    try:
        duration = get_duration(audio_path)

        entry = {
            "audio_filepath": os.path.abspath(audio_path),
            "text": item["text"],
            "duration": duration
        }
        nemo_manifest.append(entry)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Split data: 80% train, 10% validation, 10% test
train_val, test = train_test_split(nemo_manifest, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.11, random_state=42)  # 0.11 of 90% is ~10% of total

# Write manifests - one entry per line in JSONL format
def write_manifest(entries, filename):
        with open(filename, 'w') as f:
                    for entry in entries:
                                    f.write(json.dumps(entry) + '\n')

            write_manifest(train, "train_manifest.json")
write_manifest(val, "val_manifest.json")
write_manifest(test, "test_manifest.json")